# Calculating Par Yields

## Using the longest maturity bond to calculate Par Yields

In this example, we use the bond with the longest maturity in our sample, January 31$^{st}$ 2030, to estimate the par yield for each of its payment dates. The settlement date for our bond sample is January 21$^{st}$ 2025, but this date is inconvenient for our calculations.

To ensure par yield estimates are consistently calculated at six-month intervals, we base our analysis on forward prices beginning January 31$^{st}$ 2025. This approach results in "forward par yields,"; calculations performed ten days after the bond price settlement date.  Other examples coordinate the settlement dates with maturity dates.

As outlined in Chapter One, forward rates are derived from the differences in the natural logarithm of zero prices. In a similar fashion, forward zero prices are calculated by dividing the longer maturity zero price by the shorter one. In the January 31$^{st}$ 2030 example, all zero prices used for calculating the par yields are normalized by 0.998886-the January 31$^{st}$ 2025 zero price.



## Importing libraries, modules, and functions

As in earlier chapters of the volume, modules that are included in the standard Python library are imported. When necessary, other modules or libraries are installed before they are imported.

```
import sys
import requests
from types import ModuleType
from datetime import datetime, date

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
```

In [1]:
# Import OS to interact with local computer operating system
import sys
import requests
from types import ModuleType
# Import the datetime and date classes from the datetime module
from datetime import datetime, date

# Import the NumPy library for numerical operations, commonly aliased as np.
try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np

# Import the pandas library for data manipulation and analysis, aliased as pd.
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

### Adding a custom module and importing functions
The notebook utilizes the custom module, **module_basic_concepts_fixed_income**, sourced from Dropbox and named **basic_concepts_fixed_income**. This module provides three core functions to streamline the analysis and focus on financial concepts:

* **accrued_interest** (Chapter Two): This function calculates the accrued interest on a settlement date.  
  * [View Details](https://patrickjhess.github.io/Imported-Functions/accrued_interest.html#accrued-interest-is-a-helper-function-that-calculates-accrued-interest)  

* **create_payoff_matrix** (Chapter Four ): This function calculates the payoff matrix required to bootstrap the term structure with coupon bonds.  
  * [View Details](https://patrickjhess.github.io/Imported-Functions/create_payoff_matrix.html)</a>

* **bond_pay_data** (Chapter Three): Returns the payment dates and amounts.  
  * [View Details](https://patrickjhess.github.io/Imported-Functions/bond_pay_data.html#bond-pay-data-helper-function)

The module also contains various supporting helper functions, and it automatically manages the import and, if required, installation of necessary libraries and modules. This structure allows the user to concentrate fully on financial principles rather than complex Python implementation details.



```
from basic_concepts_fixed_income import (accrued_interest,
                                         create_payoff_matrix,
                                         bond_pay_data)
```





In [2]:
# Define the URL of the Python module to be downloaded from Dropbox.
# The 'dl=1' parameter in the URL forces a direct download of the file content.
url= 'https://www.dropbox.com/scl/fi/4y5hjxlfphh1ngvbgo77q/\
module_-basic_concepts_fixed_income.py?rlkey=6oxi7mgka42veaat79hcv8boz&st=87sztshr&dl=1'
module_name='basic_concepts_fixed_income'
# Send an HTTP GET request to the URL and store the server's response.
try:
  response=requests.get(url)
  # Raise an exception for bad status codes (like 404 Not Found)
  response.raise_for_status()
  module= ModuleType(module_name)
  #Code contained in response.text executed
  exec(response.text, module.__dict__)
  # Module added to sys
  sys.modules[module_name]=module
except requests.exceptions.RequestException as e:
    print(f"❌ Error: Could not fetch module from URL. {e}")
except Exception as e:
    print(f"❌ Error: Failed to execute or import the module. {e}")

# Now that 'basic_concepts_fixed_income' exists in the notebook, import the specific functions
from basic_concepts_fixed_income import (accrued_interest,
                                         create_payoff_matrix,
                                         bond_pay_data)

## Data Used To Estimate The Term Structure

The 304 U.S. Treasury securities data in the sheet 'Fidelity Data' of the Excel workbook 'Bond Price Data' are used to estimate the zero prices for five years on the settlement date January 21$^{st}$ 2025.$
^{1}$  

---
$^{1}$ The process of accessing the data and estimating the term structure is also presented in [Chapter Four: Bootstrapping The Term Structure With 304 Bonds](https://patrickjhess.github.io/Vol-3-Chap-4/Bootstrapping_The_Term_Structure_With_304_Bonds.html).


This Fidelity data is downloaded from DropBox and loaded into a Pandas DataFrame using the Panda's <font color='green'>read_excel</font> method.$^{2}$

Three arguments are passed to the method:



1.    The URL address (<font color='green'>url</font>) is required.
2.    Assigning the index column is optional. The maturity dates, located in the first column, are set as the index of the DataFrame  (<font color='green'>index_col='Maturity Date'</font>).
3.    The worksheet name defaults to the first worksheet. The worksheet's name, 'Fidelity Data', is the assigned sheet (<font color='green'>sheet_name='Fidelity Data'</font>)

The <font color='green'>display</font>  function shows the first and last five rows confirming that data has been successfully accessed.

---

 $^{2}$ <a href='https://patrickjhess.github.io/Introduction-To-Python-For-Financial-Python/An_Introduction_To_Pandas.html#dataframes-csv-and-excel-files'>Pandas method read_excel</a>

In [3]:
#The full file path.
url='https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/\
bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1'
print(f"Attempting to load data from: {url}")

#Load the data from Excel, using the first column as the index.
try:
    bond_data = pd.read_excel(url, index_col='Maturity Date',sheet_name='Fidelity Data')

    # Display the first and last 5 rows of the loaded DataFrame to verify it worked.
    display(bond_data)

except FileNotFoundError:
    print("\nERROR: File not found.")
    print("Please check that the 'folder' and 'file' variables are spelled correctly'\
' and that the file exists in that location.")

Attempting to load data from: https://www.dropbox.com/scl/fi/lgnaj41bt8o9sv5a63rr1/bond_data_jan21_2025.xlsx?rlkey=twjzkcqo0g2ahvot78518ti4x&st=ihc5feog&dl=1


,Description,Coupon,Price Bid,Price Ask,Bid Size,Ask Size
Maturity Date,,,,,,
2025-01-28,UNITED STATES TREAS BILLS ZERO CPN 0.000...,0.000,99.929,99.930,100000,100000
2025-01-30,UNITED STATES TREAS BILLS ZERO CPN 0.000...,0.000,99.906,99.907,40000,40000
2025-01-31,UNITED STATES TREAS SER U-2025 1.3750...,1.375,99.921,99.934,60000,60000
2025-01-31,UNITED STATES TREAS SER AW-2025 4.1250...,4.125,99.988,99.997,60000,60000
2025-01-31,UNITED STATES TREAS SER G-2025 2.5000...,2.500,99.953,99.965,60000,60000
...,...,...,...,...,...,...
2029-11-30,UNITED STATES TREAS SER AG-2029 4.1250...,4.125,98.910,98.914,100000,100000
2029-11-30,UNITED STATES TREAS SER S-2029 3.8750...,3.875,97.763,97.782,40000,40000
2029-12-31,UNITED STATES TREAS SER T-2029 3.8750...,3.875,97.734,97.738,65000,65000


## Estimate zero prices for 149 unique dates with 304 bonds of bond_data DataFrame

The term structure is estimated for the 149 unique payment dates of the <font color='green'>bond_data</font> DataFrame.  The <font color='green'>accrued_interest</font> calculates the transaction prices; and <font color='green'>create_payoff_matrix</font>, creates the the payoff matrix (columns for the unique dates and payoff rows for the bonds).  The Pandas <font color='green'>apply</font> add the <font color='green'>Accrued</font> column to the <font color='green'>bond_data</font> DataFrame

In [4]:
# the settlement date for the data is Jan 21, 2025
settlement=date(2025,1,21)
# apply the accrued_interest function to the DataFrame
bond_data['Accrued'] = bond_data.apply(
    lambda x: accrued_interest(
        maturity=x.name, # Assuming index is maturity
        coupon=x['Coupon'],
        settlement=settlement,
        freq=2,
        day_type='Actual/Actual'
    ), axis=1
)

# Calculate transaction prices
transaction_prices=((bond_data['Price Bid']+bond_data['Price Ask'])/2+
                    bond_data['Accrued'])

# Create payoff matrix
payoffs_304,column_dates_304=create_payoff_matrix(bond_data,settlement=settlement,freq=2)

# Estimate the present value factors/zero prices with least squares
zero_prices_304,ss2_304,rank_304,colInfo_304=np.linalg.lstsq(payoffs_304,transaction_prices)

# Maturity of the bonds in years
maturity_years=(np.array(colInfo_304,dtype='datetime64[D]')-
                np.datetime64(settlement)).astype(float)/365.25

# Continuously compounded annualized spot rates
spot_rates_304=-np.log(zero_prices_304)/np.array(maturity_years)

# Continuously compounded annualized forward rates
forward_rates_304=-np.diff(np.log(zero_prices_304))/np.diff(maturity_years)
forward_rates_304=np.insert(forward_rates_304,0,np.nan)

# Create a DataFrame of least squares present value estimates and display results
# First a dictionary
bonds_304={'Zero Prices: 304':zero_prices_304,
           'Spot Rates: 304':spot_rates_304,
           'Forward Rates: 304':forward_rates_304}

# Then a DataFrame
df_304=pd.DataFrame(bonds_304,index=column_dates_304)
display(df_304)

# Create a DataFrame of summary statistics and display results
# First a dictionary
summary_304={'Average Sum Squared Errors':ss2_304/len(bond_data),
             'Payoff Rank':rank_304,
             'Column Max Info':np.max(colInfo_304),
             'Column Min Info':np.min(colInfo_304)}
# Then a DataFrame
df_summary_304=pd.DataFrame(summary_304,index=['Summary Statistics']).T
display(df_summary_304)

/tmp/ipython-input-490079655.py:32: RuntimeWarning: divide by zero encountered in divide
  forward_rates_304=-np.diff(np.log(zero_prices_304))/np.diff(maturity_years)


,Zero Prices: 304,Spot Rates: 304,Forward Rates: 304
2025-01-28,0.999295,-0.000013,NaN
2025-01-30,0.999065,-0.000017,-0.004946
2025-01-31,0.998886,-0.000020,-0.065439
2025-02-04,0.998475,-0.000028,-0.050108
2025-02-06,0.998240,-0.000032,inf
...,...,...,...
2029-10-31,0.812875,-0.003782,inf
2029-11-15,0.809539,-0.003857,inf
2029-11-30,0.809936,-0.003848,-inf
2029-12-31,0.806965,-0.003915,inf


,Summary Statistics
Average Sum Squared Errors,0.011753
Payoff Rank,149.000000
Column Max Info,267.431046
Column Min Info,100.000000


## Get payment dates for the bond that matures on January 31$^{st}$ 2030.

Because the January 31$^{st}$ 2030 bond is used to estimate the zero prices, an estimated zero price exists for each of its semi-annual payment dates. The payment dates are calculated using the  <font color='green'>bond_pay_data</font> function.  Recall that this function relies upon the functions <font color='green'>scheduled_pay_dates</font> and <font color='green'>adjust_bond_pay_dates</font>.  All three functions were developed in Chapter Three.

The arguments of <font color='green'>bond_pay_data</font> are:


*   **settlement** assigned the date value of January 21$^{st}$ 2025.
*   **maturity** assigned the date value of January 31$^{st}$ 2030.
*   **coupon** because we are only concerned with dates, any value greater than zero may be assigned.  In our example we have assigned 0.01
*   **freq** assigned 2 for semi-annual payment frequency.

The function returns arrays of payment dates and amounts.



```
pay_dates_all,_=bond_pay_data(maturity,0.01,settlement=settlement,freq=2)
display(pay_dates_all)
```


In [7]:
# Use bond_pay_data function to return the full payment date array
settlement=date(2025,1,21)
coupon=0.01
maturity=date(2030,1,31)

# function returns tuple of two arrays (dates,amounts)
pay_data=bond_pay_data(maturity,coupon,settlement=settlement,freq=2)

# only conerned with payment dates
pay_data[0]

array([datetime.date(2025, 1, 31), datetime.date(2025, 7, 31),
       datetime.date(2026, 2, 2), datetime.date(2026, 7, 31),
       datetime.date(2027, 2, 1), datetime.date(2027, 8, 2),
       datetime.date(2028, 1, 31), datetime.date(2028, 7, 31),
       datetime.date(2029, 1, 31), datetime.date(2029, 7, 31),
       datetime.date(2030, 1, 31)], dtype=object)

## Use the first payment date as an assumed settlement date

To maintain alignment for semi-annual (six-month) par yield calculations, we use the first coupon payment date as the assumed settlement date. By asigning the settlement date as January 31, 2025 the payment date series skips the first date and begins with July 31, 2025. Using our forward zero prices allows us to  estimate par yields for equal-sized six-month periods beginning January 31, 2025.


## Calculating forward zero prices
Forward zero prices are calculated at six-month intervals, aligning with the first bond payment date which is six months after the assumed settlement date.

To determine these prices, the forward prices corresponding to the forward pay dates are identified. These prices are then divided by the spot price corresponding to the assumed settlement date.


In [11]:
# loc attribute selects the zero price estimates starting at second date
matching_zero_prices=pd.Series(df_304['Zero Prices: 304'].loc[pay_data[0][1:]])

# loc attribute selects the zero price of first date for normalization
normalizing_zero_price= df_304['Zero Prices: 304'].loc[pay_data[0][0]]

# forward zero prices calculated
forward_zero_prices=pd.Series(matching_zero_prices/normalizing_zero_price,name='Forward Zero Prices')
display(forward_zero_prices)

,Forward Zero Prices
2025-07-31,0.979138
2026-02-02,0.958690
2026-07-31,0.938935
2027-02-01,0.919015
2027-08-02,0.899549
2028-01-31,0.879982
2028-07-31,0.860395
2029-01-31,0.841154
2029-07-31,0.822730
2030-01-31,0.804344


## <font color='green'>Application: Calculate the forward zero prices for the bonds</font>



<div style="
    border-left: 12px solid green;
    line-height: 1.5;
    padding: 15px">
<br>
Settlement date determined so that all payments occur at six-month candence.
<br>


|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;December 31$^{st}$ 2029|&nbsp;&nbsp;&nbsp;4.375|
|&nbsp;&nbsp;&nbsp;October 31$^{st}$ 2029|&nbsp;&nbsp;&nbsp;4.125|



Calculate forward rates for the unique payment dates of both bonds.


 see [Chapter Five Hints: Calculate the forward zero prices for the bonds](https://patrickjhess.github.io/Hints-Results/Chapter_Five_Hints.html#application-calculate-the-forward-zero-prices-for-the-bonds), and check the [expected results here](https://patrickjhess.github.io/Hints-Results/Chapter_Five_Results.html#application-calculate-the-forward-zero-prices-for-the-bonds).

</div>

## Using the Pandas <font color='green'>cumsum</font> method to calculate par yields

Pandas <font color='green'>cumsum</font> method computes the cumulative sum for a DataFrame's columns or rows, as well as for a Series. For instance, the example below first generates a Series of integers and then outputs the result of applying this method.


In [12]:
# create the integer_series
integer_series=pd.Series([1,2,3])

# display the results of cumsum
integer_series.cumsum()

,0
0,1
1,3
2,6


## Calculating Par Yields

Calculating par yields is straightforward once bond payment dates are established. The Pandas <font color='green'>cumsum</font> method is convenient. The numerator of the par yield formula is one minus the forward zero price corresponding to the payment date, and the denominator is the cumulative sum of all forward zero prices up to and including the payment date.


In [13]:
# calculate the numerator
one_minus_forward_zero_price=1-forward_zero_prices

# cumsum to calculate the denominator
cumsum_forward_zero_prices=forward_zero_prices.cumsum()

# calculate par_yield series and name it Par Yields
par_yields=pd.Series(one_minus_forward_zero_price/cumsum_forward_zero_prices*2,name='Par Yields')
par_yields

,Par Yields
2025-07-31,0.042613
2026-02-02,0.042635
2026-07-31,0.042454
2027-02-01,0.042671
2027-08-02,0.042788
2028-01-31,0.043053
2028-07-31,0.043385
2029-01-31,0.043658
2029-07-31,0.043773
2030-01-31,0.043948


## U.S. Treasury estimates of par yields

The U.S. Treasury provides daily estimates of par yields for maturities ranging from one month to thirty years. These estimates are derived by interpolating the term structure between key dates, making it possible to determine yields for any maturity on any given date. The Treasury's 2025 estimates are downloaded as a CSV file and are benchmarked against our par yield estimates.

In [ ]:
# url address for daily par yields in 2025
url='https://home.treasury.gov/resource-center/data-chart-center/interest-rates/\
daily-treasury-rates.csv/2025/all?field_tdr_date_value=2025\
&type=daily_treasury_yield_curve&page&_format=csv'

# pandas method read_csv creates the DataFrame
us_treasury_par=pd.read_csv(url,index_col='Date')
#display to confirm DataFrame creation
us_treasury_par

,1 Mo,1.5 Month,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
Date,,,,,,,,,,,,,,
12/31/2025,3.74,3.75,3.67,3.67,3.63,3.59,3.48,3.47,3.55,3.73,3.94,4.18,4.79,4.84
12/30/2025,3.65,3.71,3.65,3.65,3.63,3.59,3.47,3.45,3.50,3.68,3.89,4.14,4.76,4.81
12/29/2025,3.69,3.70,3.70,3.68,3.66,3.59,3.48,3.45,3.51,3.67,3.88,4.12,4.75,4.80
12/26/2025,3.70,3.69,3.72,3.64,3.66,3.58,3.49,3.46,3.54,3.68,3.89,4.14,4.76,4.81
12/24/2025,3.72,3.73,3.74,3.69,3.67,3.59,3.50,3.47,3.56,3.70,3.91,4.15,4.75,4.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
01/08/2025,4.41,NaN,4.34,4.35,4.31,4.25,4.19,4.28,4.31,4.45,4.56,4.67,4.97,4.91
01/07/2025,4.42,NaN,4.35,4.35,4.31,4.24,4.19,4.30,4.33,4.46,4.57,4.67,4.97,4.91
01/06/2025,4.43,NaN,4.36,4.35,4.31,4.24,4.17,4.28,4.30,4.42,4.52,4.62,4.91,4.85


## Comparing U.S. Treasury estimates to our estimates with the January 31$^{st}$ 2030 bond.


There are two reasons that our estimtes will differ from those of the The U.S. Treasury:

1. **Settlement Date**: The U.S. Treasury's estimated settlement date is ten days earlier than the date used in our calculations, which rely on forward zero price estimates.  
2. **Maturity Dates**: Our estimates use six-month intervals between July 31, 2025, and January 31, 2030, while the U.S. Treasury reports maturity dates spanning one month to thirty years.

To compare our estimates with those of the U.S. Treasury, certain adjustments are necessary. The first step involves identifying the U.S. Treasury maturities reported for January 21, 2025. This is done by examining the index:


```
display(us_treasury_par.loc['01/21/2025'].index)
Index(['1 Mo', '1.5 Month', '2 Mo', '3 Mo', '4 Mo', '6 Mo', '1 Yr', '2 Yr',
       '3 Yr', '5 Yr', '7 Yr', '10 Yr', '20 Yr', '30 Yr'],
      dtype='object')

```

To confirm alignment with our estimates, we select and assign index values to approximate the maturity dates of our estimates:



```
ust_maturity_dates=['6 Mo','1 Yr','2 Yr','3 Yr','5 Yr']
ust_maturity_index=[date(2025,7,31),date(2026,2,2),date(2027,2,1),date(2028,1,31),date(2030,1,31)]

```
The alignment isn't perfect, but is close enough for a meaningful comparison



#### Determine U.S. Treasury maturity dates

In [ ]:
# Display maturity dates
display(us_treasury_par.loc['01/21/2025'].index)

Index(['1 Mo', '1.5 Month', '2 Mo', '3 Mo', '4 Mo', '6 Mo', '1 Yr', '2 Yr',
       '3 Yr', '5 Yr', '7 Yr', '10 Yr', '20 Yr', '30 Yr'],
      dtype='object')

#### Select U.S. Treasury maturity dates and create an index that aligns with our estimates

In [ ]:
# Set matching maturities
ust_maturity_dates=['6 Mo','1 Yr','2 Yr','3 Yr','5 Yr']

# Use approximate pay dates corresponding to our estimates
ust_maturity_index=[date(2025,7,31),date(2026,2,2),date(2027,2,1),date(2028,1,31),date(2030,1,31)]
ust_maturity_index

[datetime.date(2025, 7, 31),
 datetime.date(2026, 2, 2),
 datetime.date(2027, 2, 1),
 datetime.date(2028, 1, 31),
 datetime.date(2030, 1, 31)]

#### Get relevant U.S. Treasury data and set the index.

In [ ]:
# Select U.S. Treasury estimates for Jan. 21 2025
ust_2025_1_21=us_treasury_par.loc['01/21/2025']
# display result
print(ust_2025_1_21)

# Create a series with matching maturities
ust_maturity_par_yields=ust_2025_1_21.loc[ust_maturity_dates]

# Assign index the matching maturity dates
ust_maturity_par_yields.index=ust_maturity_index
#display result
ust_maturity_par_yields

1 Mo         4.42
1.5 Month     NaN
2 Mo         4.35
3 Mo         4.36
4 Mo         4.33
6 Mo         4.28
1 Yr         4.21
2 Yr         4.29
3 Yr         4.33
5 Yr         4.40
7 Yr         4.49
10 Yr        4.57
20 Yr        4.87
30 Yr        4.80
Name: 01/21/2025, dtype: float64


,01/21/2025
2025-07-31,4.28
2026-02-02,4.21
2027-02-01,4.29
2028-01-31,4.33
2030-01-31,4.40


In order to match the U.S. Treasury's presentation, our estimates are first scaled by multiplying them by 100 and rounding to two decimal places:

```py
scaled_par_yields = par_yields * 100.round(decimals=2)
```

To make a comparison between our estimated par yields and those from the U.S. Treasury, a DataFrame is constructed by concatenating the two par yield arrays.

The arrays are combined using the Pandas <font color='green'>concat</font> method with the following arguments:

* **Series**: The <font color='green'>scaled_par_yields</font> (our estimates) and <font color='green'>ust_maturity_par_yields</font> (U.S. Treasury's par yields) are the Series being combined.  
* **axis**: Set to <font color='green'>1</font> to add the Series as new columns.  
* **join**: Set to <font color='green'>inner</font>` to include only the intersection of dates, ensuring data is present for both series.

The resulting DataFrame is created with the following code:

```py
pd.concat([scaled_par_yields, ust_maturity_par_yields], axis=1, join='inner')
```


In [ ]:
# our estimates are scaled to be two digit percentage format
scaled_par_yields=(par_yields*100).round(decimals=2)

# use Pandas concat method to display a DataFrame of 'matching' estimates
display(pd.concat([scaled_par_yields,ust_maturity_par_yields],axis=1,join='inner'))

,Par Yields,01/21/2025
2025-07-31,4.26,4.28
2026-02-02,4.26,4.21
2027-02-01,4.27,4.29
2028-01-31,4.31,4.33
2030-01-31,4.39,4.40


## Summarizing the comparison

Although our estimates differ from the U.S. Treasury, the differences are small- with only one estimate exceeding two basis points (five basis points at the one-year mark).

We can improve the frequency of our estimates by incorporating additional payment dates in the calculation of par yields. You are asked to include three such dates in the Chapter Exercise.

Adding these dates provides greater detail, but  gaps remain outside the 149 unique dates of our estimates. Chapter Nine addresses this "missing dates" problem by introducing the Nelson-Siegel model for estimating the term structure as a continuous function. This model provides estimates for any maturity date and is conceptually similar to that used by the U.S. Treasury.


## <span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 1.0em; margin-bottom: 0em; font-style: italic;">Chapter Exercise</span>
<span style="text-align: left; color:green; font-family: 'Franklin Gothic Medium', sans-serif; margin-top: 0; margin-bottom: 0.5em; font-style: italic;
"><big></big>
</span>


<div style="background-color:LightGray;
    border-left: 12px solid green;
    font-family: 'Garamond', serif;
    font-size: 17px;
    line-height: 1.5;
    padding: 15px">
<br>
Calculate the par yields for the following bonds:


|Maturity|Coupon|
|-------|-------|
|&nbsp;&nbsp;&nbsp;December 31$^{st}$ 2029|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.375|
|&nbsp;&nbsp;&nbsp;October 31$^{th}$ 2029|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;4.125|
|&nbsp;&nbsp;&nbsp;January 31$^{th}$ 2030|&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;3.50|$
<br>


 see [Chapter Five Hints: Calculate the par yields for the following bonds.](https://patrickjhess.github.io/Hints-Results/Chapter_Five_Hints.html#chapter-exercise), and check the [expected results here](https://patrickjhess.github.io/Hints-Results/Chapter_Five_Results.html#chapter-exercise).

